<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/vqe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantum advantage in ground state energy calculation of O₂**

**Goal: Find the lowest eigenvalues of the O2 molecular Hamiltonian.**
* Method: Variational Quantum Eigensolver (VQE) for the ground state and Variational Quantum Deflation (VQD) for the excited state.
* Ansatz: AllSinglesDoubles quantum circuit to prepare the trial states for both cases

**Components**:

✅ Core VQE optimization with finite difference gradients `vqe_ground_state()`  
✅ VQD excited state calculation with SWAP test `vqd_excited_state()` (Excited state with orthogonality constraint)   
✅ Spin state verification (S² calculation)  
✅ Exact diagonalization comparison  
✅ Adaptive learning rate  
✅ Energy gap calculation in eV


In [2]:
!pip install pennylane openfermion openfermionpyscf -q

In [3]:
# @title Code
import pennylane as qml
from pennylane import qchem
import numpy as np
from functools import partial
from pennylane import AllSinglesDoubles
import warnings
warnings.filterwarnings('ignore')

# ============================================
# 1. MOLECULE SETUP
# ============================================

def setup_molecule(active_electrons=2, active_orbitals=3, r=2.30):
    """Setup O2 molecule and return Hamiltonian"""
    symbols = ['O', 'O']
    coordinates = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, r]])
    molecule = qchem.Molecule(symbols, coordinates)

    H, qubits = qchem.molecular_hamiltonian(
        molecule,
        active_electrons=active_electrons,
        active_orbitals=active_orbitals,
        method='openfermion'
    )

    print(f"System: {qubits} qubits, {len(H)} Hamiltonian terms")
    return H, qubits, molecule

# ============================================
# 2. GROUND STATE VQE
# ============================================

def vqe_ground_state(H, qubits, active_electrons, max_iterations=150):
    """Calculate ground state using VQE with gradient descent"""

    # Setup excitations and initial state
    singles, doubles = qchem.excitations(active_electrons, qubits)
    hf_state = qchem.hf_state(active_electrons, qubits)
    n_params = len(singles) + len(doubles)

    print(f"Number of parameters: {n_params}")

    # Create device and circuit
    dev = qml.device("lightning.qubit", wires=qubits)

    @qml.qnode(dev)
    def circuit(params):
        qml.AllSinglesDoubles(params, range(qubits), hf_state, singles, doubles)
        return qml.expval(H)

    # Initialize parameters
    params = np.random.randn(n_params) * 0.05

    # Optimization parameters
    learning_rate = 0.4
    epsilon = 0.01  # For finite differences
    best_energy = float('inf')
    best_params = params.copy()

    print("\nOptimizing ground state...")
    for step in range(max_iterations):
        # Calculate current energy
        current_energy = float(circuit(params))

        # Calculate gradient using finite differences
        grad = np.zeros_like(params)
        for i in range(len(params)):
            params_plus = params.copy()
            params_minus = params.copy()
            params_plus[i] += epsilon
            params_minus[i] -= epsilon

            grad[i] = (circuit(params_plus) - circuit(params_minus)) / (2 * epsilon)

        # Update parameters
        params = params - learning_rate * grad

        # Track best result
        if current_energy < best_energy:
            best_energy = current_energy
            best_params = params.copy()

        # Adaptive learning rate
        if step % 40 == 0 and step > 0:
            learning_rate *= 0.8

        # Progress report
        if step % 30 == 0:
            print(f"  Step {step}: Energy = {current_energy:.8f} Ha")

        # Early stopping
        if np.linalg.norm(grad) < 1e-5:
            print(f"  Converged at step {step}")
            break

    return best_params, best_energy

# ============================================
# 3. EXCITED STATE VQD
# ============================================

def vqd_excited_state(H, qubits, active_electrons, ground_params, beta=2.0, max_iterations=100):
    """Calculate first excited state using VQD with SWAP test"""

    singles, doubles = qchem.excitations(active_electrons, qubits)
    hf_state = qchem.hf_state(active_electrons, qubits)
    n_params = len(singles) + len(doubles)

    # Device for energy calculation
    dev_energy = qml.device("lightning.qubit", wires=qubits)

    @qml.qnode(dev_energy)
    def energy_circuit(params):
        qml.AllSinglesDoubles(params, range(qubits), hf_state, singles, doubles)
        return qml.expval(H)

    # Ansatz definition for SWAP test
    @partial(qml.transforms.decompose, max_expansion=1)
    def ansatz(theta, wires):
        singles_list, doubles_list = qml.qchem.excitations(active_electrons, qubits)
        singles_list = [[wires[i] for i in single] for single in singles_list]
        doubles_list = [[wires[i] for i in double] for double in doubles_list]
        AllSinglesDoubles(theta, wires, hf_state, singles_list, doubles_list)

    # Device for SWAP test (needs 2*qubits + 1 wires)
    dev_swap = qml.device("lightning.qubit", wires=2*qubits+1)

    @qml.qnode(dev_swap)
    def swap_test(params):
        # Prepare ground state on qubits 1 to qubits
        ansatz(ground_params, range(1, qubits+1))
        # Prepare trial state on qubits qubits+1 to 2*qubits
        ansatz(params, range(qubits + 1, 2 * qubits + 1))

        # SWAP test circuit
        qml.Hadamard(wires=0)
        for i in range(qubits):
            qml.CSWAP(wires=[0, 1 + i + qubits, 1 + i])
        qml.Hadamard(wires=0)

        return qml.expval(qml.Z(0))

    # Initialize parameters orthogonal to ground state
    params = np.random.randn(n_params) * 0.1
    # Gram-Schmidt orthogonalization
    params = params - np.dot(params, ground_params) * ground_params / (np.linalg.norm(ground_params)**2 + 1e-8)
    params = params * 0.1

    # Optimization parameters
    learning_rate = 0.2
    epsilon = 0.01

    print("\nOptimizing excited state...")
    for step in range(max_iterations):
        # Calculate energy and overlap
        energy = float(energy_circuit(params))
        overlap = float(swap_test(params))
        cost = energy + beta * overlap

        # Calculate gradient using finite differences
        grad = np.zeros_like(params)
        for i in range(len(params)):
            params_plus = params.copy()
            params_minus = params.copy()
            params_plus[i] += epsilon
            params_minus[i] -= epsilon

            energy_plus = energy_circuit(params_plus)
            energy_minus = energy_circuit(params_minus)
            overlap_plus = swap_test(params_plus)
            overlap_minus = swap_test(params_minus)

            cost_plus = energy_plus + beta * overlap_plus
            cost_minus = energy_minus + beta * overlap_minus

            grad[i] = (cost_plus - cost_minus) / (2 * epsilon)

        # Update parameters
        params = params - learning_rate * grad

        # Progress report
        if step % 20 == 0:
            print(f"  Step {step}: Energy = {energy:.6f} Ha, Overlap = {overlap:.4f}, Cost = {cost:.6f}")

        # Adaptive learning rate
        if step % 40 == 0 and step > 0:
            learning_rate *= 0.8

    final_energy = float(energy_circuit(params))
    final_overlap = float(swap_test(params))

    print(f"\nFinal excited state: Energy = {final_energy:.6f} Ha, Overlap = {final_overlap:.4f}")

    return params, final_energy

# ============================================
# 4. ANALYSIS FUNCTIONS
# ============================================

def calculate_spin(params, qubits, active_electrons):
    """Calculate S² expectation value"""
    singles, doubles = qchem.excitations(active_electrons, qubits)
    hf_state = qchem.hf_state(active_electrons, qubits)
    S2 = qchem.spin2(active_electrons, qubits)

    dev = qml.device("lightning.qubit", wires=qubits)

    @qml.qnode(dev)
    def spin_circuit(params):
        qml.AllSinglesDoubles(params, range(qubits), hf_state, singles, doubles)
        return qml.expval(S2)

    return float(spin_circuit(params))

def get_exact_energies(H, n_states=10):
    """Calculate exact energies using diagonalization"""
    eigenvalues = np.sort(np.linalg.eigvals(H.matrix()).real)
    print("\nExact eigenvalues:")
    for i in range(min(n_states, len(eigenvalues))):
        print(f"  State {i}: {eigenvalues[i]:.8f} Ha")
    return eigenvalues

# ============================================
# 5. MAIN WORKFLOW
# ============================================

def run_vqd(active_electrons=2, active_orbitals=3, r=2.30, compare_exact=True):
    """Complete VQD workflow for ground and excited states"""

    print("="*60)
    print(f"VQD for O2 Molecule")
    print(f"Active space: {active_electrons} electrons, {active_orbitals} orbitals")
    print("="*60)

    # Setup molecule
    H, qubits, molecule = setup_molecule(active_electrons, active_orbitals, r)

    # Calculate ground state
    print("\n" + "="*60)
    print("Ground State Calculation")
    print("="*60)
    ground_params, ground_energy = vqe_ground_state(H, qubits, active_electrons)
    print(f"\nGround state energy: {ground_energy:.8f} Ha")

    # Calculate spin
    s2_ground = calculate_spin(ground_params, qubits, active_electrons)
    print(f"Ground state S² = {s2_ground:.6f}")

    # Calculate excited state
    print("\n" + "="*60)
    print("Excited State Calculation")
    print("="*60)
    excited_params, excited_energy = vqd_excited_state(H, qubits, active_electrons, ground_params)
    print(f"\nExcited state energy: {excited_energy:.8f} Ha")

    # Energy gap
    gap_eV = (excited_energy - ground_energy) * 27.2114  # Hartree to eV
    print(f"Energy gap: {gap_eV:.4f} eV")

    # Calculate spin for excited state
    s2_excited = calculate_spin(excited_params, qubits, active_electrons)
    print(f"Excited state S² = {s2_excited:.6f}")

    # Compare with exact if requested
    if compare_exact and qubits <= 10:
        print("\n" + "="*60)
        print("Comparison with Exact Diagonalization")
        print("="*60)
        exact_energies = get_exact_energies(H)

        error_ground = abs(ground_energy - exact_energies[0]) * 1000
        error_excited = abs(excited_energy - exact_energies[1]) * 1000

        print(f"\nErrors:")
        print(f"  Ground state: {error_ground:.3f} mHa")
        print(f"  Excited state: {error_excited:.3f} mHa")

    return {
        'ground_params': ground_params,
        'ground_energy': ground_energy,
        'excited_params': excited_params,
        'excited_energy': excited_energy,
        'energy_gap_eV': gap_eV,
        's2_ground': s2_ground,
        's2_excited': s2_excited
    }

# ============================================
# EXECUTION
# ============================================

if __name__ == "__main__":
    # Run for small system
    results = run_vqd(active_electrons=2, active_orbitals=3, r=2.30)

    print("\n" + "="*60)
    print("Summary")
    print("="*60)
    print(f"Ground state: {results['ground_energy']:.8f} Ha (S² = {results['s2_ground']:.4f})")
    print(f"Excited state: {results['excited_energy']:.8f} Ha (S² = {results['s2_excited']:.4f})")
    print(f"Energy gap: {results['energy_gap_eV']:.4f} eV")

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


VQD for O2 Molecule
Active space: 2 electrons, 3 orbitals
System: 6 qubits, 34 Hamiltonian terms

Ground State Calculation
Number of parameters: 8

Optimizing ground state...
  Step 0: Energy = -147.54567787 Ha
  Step 30: Energy = -147.55929496 Ha
  Step 60: Energy = -147.56515438 Ha
  Step 90: Energy = -147.56923975 Ha
  Step 120: Energy = -147.57202041 Ha

Ground state energy: -147.57385373 Ha
Ground state S² = 0.019812

Excited State Calculation

Optimizing excited state...
  Step 0: Energy = -147.550710 Ha, Overlap = 0.7935, Cost = -145.963620
  Step 20: Energy = -147.531024 Ha, Overlap = 0.0010, Cost = -147.528944
  Step 40: Energy = -147.533117 Ha, Overlap = 0.0001, Cost = -147.533010
  Step 60: Energy = -147.533530 Ha, Overlap = 0.0001, Cost = -147.533405
  Step 80: Energy = -147.533792 Ha, Overlap = 0.0001, Cost = -147.533664

Final excited state: Energy = -147.533955 Ha, Overlap = 0.0001

Excited state energy: -147.53395520 Ha
Energy gap: 1.0857 eV
Excited state S² = 0.007704
